# Regressão Softmax com dados do MNIST

O objetivo deste notebook é ilustrar o uso do mesmo código matricial desenvolvido para a classificação das Flores Íris, porém agora com o problema de classificação de dígitos manuscritos utilizando o dataset MNIST.


## Importação das bibliotecas

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
from torch.autograd import Variable

## Funções já discutidas em notebooks anteriores

In [2]:
def predict(model, inputs):
    outputs = model(Variable(inputs))
    _, predicts = torch.max(outputs, 1)
    
    return predicts.data.numpy()

def getAccuracy(model, inputs, targets):
    outputs = model(Variable(inputs))
    _, predicts = torch.max(outputs, 1)

    predicts = predicts.data.numpy()
    targets = targets.numpy()
    
    accuracy = (predicts == targets).mean()
    return accuracy

## Carregamento dos dados do MNIST

In [3]:
dataset_dir = '/data/datasets/MNIST/'

x_train, y_train = torch.load(dataset_dir + 'processed/training.pt')
x_test,  y_test  = torch.load(dataset_dir + 'processed/test.pt')

print("Amostras para treinamento:", x_train.size(0))
print("Amostras para validação   ", x_test.size(0))

print("\nDimensões dos dados das imagens:   ", x_train.size())
print("Valores mínimo e máximo dos pixels:", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:        ", type(x_train))
print("Tipo das classes das imagens:      ", type(y_train))

Amostras para treinamento: 60000
Amostras para validação    10000

Dimensões dos dados das imagens:    torch.Size([60000, 28, 28])
Valores mínimo e máximo dos pixels: 0 255
Tipo dos dados das imagens:         <class 'torch.ByteTensor'>
Tipo das classes das imagens:       <class 'torch.LongTensor'>


## Carregamento, normalização e seleção dos dados do MNIST

Neste exemplo utilizaremos 500 amostras de treinamento e 100 amostras de validação.

In [4]:
x_train = x_train.float()
x_test = x_test.float()

x_train = x_train / 255.
x_test = x_test / 255.

if True:
    n_samples_train = 1000
    n_samples_test  = 500

    x_train = x_train[:n_samples_train]
    y_train = y_train[:n_samples_train]
    x_test  = x_test[:n_samples_test]
    y_test  = y_test[:n_samples_test]

print("Amostras para treinamento:", x_train.size(0))
print("Amostras para validação   ", x_test.size(0))

print("\nDimensões dos dados das imagens:   ", x_train.size())
print("Valores mínimo e máximo dos pixels:", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:        ", type(x_train))
print("Tipo das classes das imagens:      ", type(y_train))

Amostras para treinamento: 1000
Amostras para validação    500

Dimensões dos dados das imagens:    torch.Size([1000, 28, 28])
Valores mínimo e máximo dos pixels: 0.0 1.0
Tipo dos dados das imagens:         <class 'torch.FloatTensor'>
Tipo das classes das imagens:       <class 'torch.LongTensor'>


## Visualizando os dados

In [5]:
n_samples = 24

# cria um DataLoader temporario para pegar um batch de 'n_samples' imagens de treinamento
temp_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                              batch_size=n_samples,
                                              shuffle=True)

# pega um batch de imagens
image_batch, labels = next(iter(temp_dataloader))

# cria um grid com as imagens
grid = torchvision.utils.make_grid(image_batch, normalize=True, pad_value=1.0, padding=1)

plt.figure(figsize=(15, 10))
plt.imshow(grid.numpy().transpose(1, 2, 0))
plt.axis('off')
plt.show()

AttributeError: module 'torch.utils' has no attribute 'data'

## Visualizando uma imagem com o matplotlib

In [ ]:
image, target = train_dataset[0]

plt.imshow(image.numpy().reshape(28,28), cmap='gray')
print('class:', target)

## Treinamento

### Inicialização dos parâmetros

In [ ]:
epochs = 100
learningRate = 0.5

# Cria um DataLoader de somente um batch
train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=16,
                                               shuffle=True)

# Pega todas as imagens de uma vez


# Cria uma operação linear com entrada de tamanho 28*28 e saída com 10 neurônios (classes)
# O objeto criado armazenará os pesos
model = torch.nn.Linear(28*28, 10)

# Utilizaremos CrossEntropyLoss como função de perda
criterion = torch.nn.CrossEntropyLoss(size_average=True)
criterion2 = torch.nn.CrossEntropyLoss(size_average=False)

# Nosso otomizador será SDG
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

### Laço de treinamento dos pesos

In [ ]:
losses = []

for i in range(epochs):
    loss_sum = 0
    loss2_sum= 0
    
    for batch in train_dataloader:
        # separa os dados do batch
        input_data, targets_data = batch
        # Transforma a entrada para uma dimensão
        input_data = input_data.view(-1, 28*28)
    
        # calcula a saída da operação linear
        output = model(Variable(input_data))

        # calcula a perda
        loss = criterion(output, Variable(targets_data))
        loss_sum += loss.data[0] * targets_data.size(0)
        loss2 = criterion2(output, Variable(targets_data))
        loss2_sum += loss2
        
        # zero, backpropagation, ajusta parâmetros pelo gradiente descendente
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    epoch_loss = loss_sum / len(train_dataset)
    epoch2_loss = loss2_sum/ len(train_dataset)
    losses.append(epoch_loss)
    

In [ ]:
print('Final loss:', epoch_loss, epoch2_loss.data[0])

In [ ]:
1/(0.1506661695893854/ 2.3205604553222656)

## Visualizando gráfico de perda durante o treinamento

In [ ]:
plt.plot(losses)

## Avaliando a acurácia tanto no conjunto de treinamento como no conjunto de testes

In [ ]:
def getAccuracy(model, inputs, targets):
    outputs = model(Variable(inputs))
    _, predicts = torch.max(outputs, 1)

    predicts = predicts.data.numpy()
    targets = targets.numpy()
    
    accuracy = (predicts == targets).mean()
    return accuracy

train_dataloader = torch.utils.data.DataLoader(train_dataset, 
                                              batch_size=len(train_dataset),
                                              shuffle=False)
train_input, targets_data = next(iter(train_dataloader))
train_input = train_input.view(-1, 28*28)

print('Training Accuracy: ', getAccuracy(model, train_input, targets_data))



test_dataloader = torch.utils.data.DataLoader(test_dataset, 
                                              batch_size=len(test_dataset),
                                              shuffle=False)
test_input, test_labels = next(iter(test_dataloader))
test_input = test_input.view(-1, 28*28)

print('Test Accuracy: ', getAccuracy(model, test_input, test_labels))

## Matriz de confusão com dados de treinamento e teste

In [ ]:
print('Matriz de confusão (Treino):')
pd.crosstab(predict(model, train_input), targets_data.numpy())

In [ ]:
print('Matriz de confusão (Teste):')
pd.crosstab(predict(model, test_input), test_labels.numpy())

## Visualizando a matriz de pesos treinados

Observe que a matriz de peso treinado para cada classe mostra a importância dos pesos associados aos caracteres de cada classe.

In [ ]:
weights = model.state_dict()['weight']
print('weights:', weights.shape)

bias = model.state_dict()['bias']
print('bias:   ', bias.shape)

# Visualizando pesos da classe 3
plt.imshow(weights[3, :].numpy().reshape((28,28)),cmap = 'gray')
plt.show()

## Visualizando os pesos de todas as classes

In [ ]:
# cria um grid com as imagens
grid = torchvision.utils.make_grid(weights.view(-1, 1, 28, 28), normalize=True, pad_value=1.0, padding=1, nrow=10)

plt.figure(figsize=(15, 10))
plt.imshow(grid.numpy().transpose(1, 2, 0))
plt.axis('off')
plt.show()

## Diagrama da regressão softmax com visualização dos pesos W

<img src="../figures/RegressaoSoftmaxArgmaxNMIST.png",width = 400>


# Atividades

## Exercícios

1. Na configuração da figura acima, mostre os valores de z0 até z9, os valores das probabilidades y_oh_hat e o y_hat, quando a rede recebe como entrada a nona amostra que contém o manuscrito do dígito '4':

In [ ]:
image, target = train_dataset[9]

plt.imshow(image.numpy().reshape(28,28), cmap='gray')
print('class:', target)

2. Insira código no laço do treinamento para que no final de cada época, 
seja impresso: o número da época e a perda e a acurácia
3. Insira código no laço de treinamento para que seja impresso a contagem
de mini-batches ocorrida em cada época.

## Perguntas

1. Qual é o tamanho do mini-batch?
2. Em uma época, quantos mini-batches existem?
3. Por que no treino, a acurácia é 100%, mas no teste foi de 84,5%
4. Por que no treino, a acurácia é 100%, porém a função de perda final não é zero, mas sim 0,015 ?
5. O que se deve fazer para que a avaliação no conjunto de teste seja melhorado?

## Conclusões sobre os experimentos deste notebook
